In [1]:
import treeflow.libsbn
import treeflow.model
import numpy as np

newick_file = 'demo-data/dengue.newick'
fasta_file = 'demo-data/dengue.fasta'

inst = treeflow.libsbn.get_instance(newick_file)
q_tree = treeflow.model.construct_tree_approximation(newick_file, inst=inst)

In [2]:
q_tree.sample()

{'topology': {'parent_indices': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([17, 17, 18, 19, 19, 20, 22, 23, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         18, 21, 20, 21, 22, 32, 24, 25, 26, 27, 28, 29, 30, 31, 32])>},
 'heights': <tf.Tensor: shape=(33,), dtype=float32, numpy=
 array([ 10.00009 ,  16.00008 ,  16.00009 ,  30.00009 ,  10.00011 ,
         38.0001  ,  31.00009 ,  12.00005 ,  11.00005 ,  13.00006 ,
         10.00005 ,   8.00005 ,   0.      ,  15.00005 ,   9.00005 ,
         17.00005 ,  18.00005 ,  28.507296,  86.97663 ,  52.00375 ,
         58.61163 , 132.61856 , 147.09761 ,  12.63696 ,  13.148711,
         13.399259,  13.446976,  13.450745,  16.130322,  16.45659 ,
         47.743423,  48.965336, 149.92657 ], dtype=float32)>}

In [3]:
chain_bijector = q_tree.height_distribution.node_height_distribution.bijector
height_to_ratio_bijector = chain_bijector.ratio_bijector
real_to_ratio_bijector = chain_bijector.to_ratio_bijector
init_heights = np.array(height_to_ratio_bijector.tree.node_heights)
base_distribution = q_tree.height_distribution.node_height_distribution.distribution
ratio_sample = real_to_ratio_bijector.forward(base_distribution.sample())
height_to_ratio_bijector.forward(ratio_sample), init_heights

(<tf.Tensor: shape=(16,), dtype=float32, numpy=
 array([26.448656 , 29.608915 , 33.911022 , 37.23276  , 34.81719  ,
        23.277449 , 12.619947 , 13.302016 , 13.857838 , 14.881178 ,
        14.889234 , 15.5048485, 15.545942 , 18.184803 , 18.219877 ,
        23.220041 ], dtype=float32)>,
 array([ 10.        ,  16.        ,  16.        ,  30.        ,
         10.        ,  38.        ,  31.        ,  12.        ,
         11.        ,  13.        ,  10.        ,   8.        ,
          0.        ,  15.        ,   9.        ,  17.        ,
         18.        ,  28.50729511,  86.97663387,  52.00374891,
         58.61162827, 132.61856657, 147.09761809,  12.63695975,
         13.14871087,  13.39925901,  13.44697533,  13.45074424,
         16.13032169,  16.45659145,  47.74342179,  48.96533664,
        149.92657471]))

In [4]:
import tensorflow as tf

with tf.GradientTape() as t:
    t.watch(ratio_sample)
    heights = height_to_ratio_bijector.forward(ratio_sample)
    res = tf.reduce_sum(heights * heights)
    
t.gradient(res, ratio_sample)

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ 1767.6508  ,  2628.5437  ,   634.99524 ,  -299.1271  ,
        -826.0055  , -1234.8271  ,    22.573204,    32.886696,
          62.076458,    96.65865 ,   217.3868  ,    53.682484,
         164.80824 ,    72.58271 ,   245.53429 ,   190.28377 ],
      dtype=float32)>